In [1]:
from funciones import Functions


#section_titles = [(id, Titulo),
#                  (id, Titulo)]
section_titles = [("login", "Login Section"),
              ("local_statistics", "Local Statistics"),
              ("global_statistics", "Global Statistics"),
              ("create_records", "Create Records"),
              ("update_delete_records", "Update Delete Records")]

Functions().create_navBar(section_titles)

HTML(value='\n        <style>\n            #navbar {\n              background-color: #FF5733; /* Color de fon…

# Bienvenido al Sistema Analítico Hotelero

**Descripcion para describir la pagina web**

---

In [2]:
import requests
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt

output_area = widgets.Output()
# Widget para mostrar la información almacenada en las cookies
cookie_info = widgets.Textarea(value='', placeholder='Información de las cookies aparecerá aquí...', 
                               description='Cookies:', disabled=False, layout={'width': '400px', 'height': '100px'})

username_input = widgets.Text(
    value='',
    placeholder='Ingrese su nombre de usuario',
    description='Usuario:',
    disabled=False
)

password_input = widgets.Password(
    placeholder='Ingrese su contraseña',
    description='Contraseña:',
    disabled=False
)

login_button = widgets.Button(
    description='Iniciar sesión',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Enviar',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

session = requests.Session()

@output_area.capture(clear_output=True)
def on_login_button_clicked(b):
    # Aquí usarías `requests` para enviar los datos al servidor
    url = 'http://127.0.0.1:5000/login/in/frontend'
    data = {
        'username': username_input.value,
        'password': password_input.value
    }
    response = session.post(url, json=data, verify=False)  # `session` debería ser una instancia de `requests.Session()` ya creada
    
    # Ahora, maneja la respuesta
    if response.status_code == 200:
        print("Login exitoso")
        # Actualizar la información de las cookies
        cookie_info.value = str(session.cookies.get_dict())
        # Aquí podrías actualizar el notebook o hacer algo con la respuesta
    else:
        print("Error en el login:", response.text)


login_button.on_click(on_login_button_clicked)
display(username_input, password_input, login_button, output_area)

# url = 'http://127.0.0.1:5000/login/in/frontend'
# body = {
#     'username': 'nauten1',
#     'password': 'mQ056w_b'
# }
# respuesta = session.post(url, json=body,verify=False)  # Asumiendo que aún estás en un entorno de desarrollo
# print("Simulando un form de html, el usuario ingresa su username y su password", "body: ", body)
# print("requests.post(http://127.0.0.1:5000/login/in/frontend)")
# print("Estado de la respuesta:", respuesta.status_code)
# print("Contenido de la respuesta:", respuesta.json())  
# print("Informacion del employee guardados en las cookies del servidor")



Text(value='', description='Usuario:', placeholder='Ingrese su nombre de usuario')

Password(description='Contraseña:', placeholder='Ingrese su contraseña')

Button(description='Iniciar sesión', icon='check', style=ButtonStyle(), tooltip='Enviar')

Output()

In [3]:
#Seccion para hacer log in
funcs = Functions()
funcs.create_styled_section(section_titles[0][0], section_titles[0][1])
display(cookie_info.value)

HTML(value='\n        <style>\n        #login {\n          color: white; /* Cambia el color del texto */\n    …

''

In [4]:
#seccion para mostrar los local stats
funcs = Functions()
funcs.create_styled_section(section_titles[1][0], section_titles[1][1])
display(cookie_info)

HTML(value='\n        <style>\n        #local_statistics {\n          color: white; /* Cambia el color del tex…

Textarea(value='', description='Cookies:', layout=Layout(height='100px', width='400px'), placeholder='Informac…

In [5]:
#seccion para mostrar los global stats
funcs = Functions()
funcs.create_styled_section(section_titles[2][0], section_titles[2][1])
display(cookie_info)



HTML(value='\n        <style>\n        #global_statistics {\n          color: white; /* Cambia el color del te…

Textarea(value='', description='Cookies:', layout=Layout(height='100px', width='400px'), placeholder='Informac…

In [6]:
#seccion para crear nuevos recors
funcs = Functions()
funcs.create_styled_section(section_titles[3][0], section_titles[3][1])
display(cookie_info)





HTML(value='\n        <style>\n        #create_records {\n          color: white; /* Cambia el color del texto…

Textarea(value='', description='Cookies:', layout=Layout(height='100px', width='400px'), placeholder='Informac…

In [7]:
url = 'http://127.0.0.1:5000/most/reservation'
data = {
    "eid": 3
}
response = session.get(url, json=data, verify=False)
# Verifica si la solicitud fue exitosa
if response.status_code == 200:
    print("Respuesta exitosa")
    data_json = response.json()
    # Convierte los datos JSON en un DataFrame de pandas
    df = pd.DataFrame(data_json)

    # Configura el tamaño de la figura
    plt.figure(figsize=(10, 6))

    # Crea un gráfico de barras
    bars = plt.bar(df['hname'], df['reservation_count'], color='skyblue')

    # Añade título y etiquetas a los ejes
    plt.title('Top 10% de Hoteles con Más Reservaciones')
    plt.xlabel('Nombre del Hotel')
    plt.ylabel('Número de Reservaciones')

    # Rotar las etiquetas del eje X para mejor legibilidad
    plt.xticks(rotation=45, ha="right")

    # Añade los valores exactos encima de las barras
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2.0, height, '%d' % int(height), ha='center', va='bottom')

    # Ajusta el layout para asegurar que las etiquetas no se corten
    plt.tight_layout()

    # Muestra el gráfico
    plt.show()
else:
    print("Error en la respuesta:", response.status_code)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
#sexxion para borrar y actualizar nuevos records
funcs = Functions()
funcs.create_styled_section(section_titles[4][0], section_titles[4][1])
display(cookie_info)